In [2]:
from dotenv import load_dotenv, find_dotenv; load_dotenv(find_dotenv())

True

In [3]:
import json, tqdm, time

In [4]:
with open('data/recipe_urls.json', 'r') as f:
    recipe_urls = json.load(f)

In [4]:
# for recipes in recipe_urls.values():
#     for recipe in recipes:
#         # print(recipe['url'].split('/')[-2])
#         recipe['id'] = recipe['url'].split('/')[-2]
#         # break
#     # break
# with open('data/recipe_urls.json', 'w') as f:
#     json.dump(recipe_urls, f, ensure_ascii=False, indent=4)

In [5]:
from extract_from_html import recipe_chain, load_doc
from recipe_schema import RecipeCreate
from typing import Optional
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

In [6]:
class RecipeWithURL(RecipeCreate):
    url: Optional[str]
    image_url: Optional[str]

In [7]:
chain = RunnableLambda(load_doc) | {'web_text': RunnablePassthrough()} | recipe_chain

In [8]:
recipes = []
for genre, rs in recipe_urls.items():
    for r in rs:
        r['genre'] = genre
        recipes.append(r)

In [9]:
len(recipes)

700

In [10]:
recipes[0]

{'url': 'https://www.lettuceclub.net/recipe/dish/23993/',
 'image': 'https://www.lettuceclub.net/i/R1/img/dish/1/S20170310053001A_002.jpg?w=350',
 'title': '電子レンジで簡単温泉卵',
 'id': '23993',
 'genre': '和食'}

In [11]:
errors = []
for recipe in tqdm.tqdm(recipes):
    try:
        res = chain.invoke(recipe['url'])
        r = RecipeWithURL(url=recipe['url'], image_url=recipe['image'], **res).dict()
        r['name'] = recipe['title']
        r['provider'] = 'lettuce_club'
        r['id'] = recipe['id']
        r['genre'] = recipe['genre']
        with open(f"data/recipes/{r['id'].zfill(10)}.json", 'w') as f:
            f.write(json.dumps(r, indent=4, ensure_ascii=False))
        time.sleep(1.)
    except Exception as e:
        errors.append(recipe)
        print('Error:', e, recipe['url'])
    # break

  6%|▌         | 39/700 [05:12<1:43:13,  9.37s/it]

Error: {'url': 'https://www.lettuceclub.net/recipe/dish/12407/', 'image': 'https://www.lettuceclub.net/i/R1/img/dish/1/S20090925034001A_000.png?w=350', 'title': '基本のおいなりさん【by 井上和子さん】', 'id': '12407', 'genre': '和食'}


  7%|▋         | 47/700 [06:19<1:34:38,  8.70s/it]

Error: {'url': 'https://www.lettuceclub.net/recipe/dish/7538/', 'image': 'https://www.lettuceclub.net/i/R1/img/dish/1/S20130510032002A_000.png?w=350', 'title': '海鮮かた焼きそば【by コウケンテツさん】', 'id': '7538', 'genre': '和食'}


  8%|▊         | 59/700 [07:59<1:25:22,  7.99s/it]

...

 80%|████████  | 560/700 [1:21:27<20:53,  8.95s/it]

Error: {'url': 'https://www.lettuceclub.net/recipe/dish/2568/', 'image': 'https://www.lettuceclub.net/i/R1/img/dish/1/2568_0_0.jpg?w=350', 'title': 'かつおのガーリックスパゲッティ', 'id': '2568', 'genre': '韓国風'}


 84%|████████▍ | 590/700 [1:26:25<18:49, 10.27s/it]

Error: {'url': 'https://www.lettuceclub.net/recipe/dish/3445/', 'image': 'https://www.lettuceclub.net/i/R1/img/dish/1/3445_0_0.jpg?w=350', 'title': 'いわしのイタリアンソテー', 'id': '3445', 'genre': '韓国風'}


100%|██████████| 700/700 [1:41:47<00:00,  8.72s/it]


In [12]:
len(errors)

23

In [13]:
with open('data/errors.json', 'w') as f:
    f.write(json.dumps(errors, indent=4, ensure_ascii=False))

In [ ]:
with open('data/errors.json', 'r') as f:
    errors = json.load(f)

In [15]:
errors2 = []
for recipe in tqdm.tqdm(errors):
    try:
        res = chain.invoke(recipe['url'])
        r = RecipeWithURL(url=recipe['url'], image_url=recipe['image'], **res).dict()
        r['name'] = recipe['title']
        r['provider'] = 'lettuce_club'
        r['id'] = recipe['id']
        r['genre'] = recipe['genre']
        with open(f"data/recipes/{r['id'].zfill(10)}.json", 'w') as f:
            f.write(json.dumps(r, indent=4, ensure_ascii=False))
        time.sleep(1.)
    except Exception as e:
        errors2.append(recipe)
        print('Error:', e, recipe['url'])

  9%|▊         | 2/23 [00:29<04:51, 13.86s/it]

Error: 1 validation error for RecipeWithURL
ingredients -> 14 -> quantity
  field required (type=value_error.missing) https://www.lettuceclub.net/recipe/dish/7538/


 13%|█▎        | 3/23 [00:34<03:22, 10.15s/it]

Error: 2 validation errors for RecipeWithURL
ingredients -> 1 -> quantity
  field required (type=value_error.missing)
ingredients -> 2 -> quantity
  field required (type=value_error.missing) https://www.lettuceclub.net/recipe/dish/4420/


 17%|█▋        | 4/23 [00:42<02:55,  9.23s/it]

...

Error: 2 validation errors for RecipeWithURL
ingredients -> 1 -> quantity
  field required (type=value_error.missing)
ingredients -> 6 -> quantity
  field required (type=value_error.missing) https://www.lettuceclub.net/recipe/dish/4945/


 74%|███████▍  | 17/23 [02:40<00:53,  9.00s/it]

Error: 1 validation error for RecipeWithURL
ingredients -> 6 -> quantity
  field required (type=value_error.missing) https://www.lettuceclub.net/recipe/dish/17978/


 74%|███████▍  | 17/23 [02:49<00:59,  9.95s/it]


KeyboardInterrupt: 